# TODO  привести в нормальный вид

In [103]:
import os
import sys


In [104]:
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'



In [105]:
spark_home = os.environ.get('SPARK_HOME', None)

In [106]:

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

In [107]:

sys.path.insert(0, os.path.join(spark_home, 'python'))

In [108]:
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [109]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [110]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [111]:
from pyspark import SparkContext

# config = SparkConf()
# config.set("spark.app.name", "Sergey Grishaev Spark RDD app")

# sc = SparkContext(conf=config)

In [112]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [113]:
sc

<SparkContext master=yarn appName=test>

In [114]:
rdd_data = sc.textFile("/labs/laba01/ml-100k/u.data")
rdd_data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [115]:
all_films = (rdd_data.map(lambda x: (x.split("\t")[2]))
         .countByKey()
         )
all_films

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [116]:
rdd_data.groupBy(lambda x: x[2]).count()

11

In [117]:
my_film = (rdd_data.filter(lambda x: "\t318" in x).map(lambda x: (x.split("\t")[2]))
         .countByKey()
         )
my_film

defaultdict(int, {'5': 186, '4': 79, '3': 23, '2': 6, '1': 4})

In [118]:
(rdd_data.filter(lambda x: "318" in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(lambda x, y: x + y)
         .collect())

[('48', 3),
 ('346', 1),
 ('14', 3),
 ('4', 2),
 ('694', 1),
 ('188', 2),
 ('408', 3),
 ('724', 1),
 ('163', 1),
 ('712', 2),
 ('219', 2),
 ('514', 2),
 ('502', 1),
 ('121', 3),
 ('100', 2),
 ('272', 1),
 ('307', 1),
 ('8', 1),
 ('1204', 1),
 ('1057', 1),
 ('40', 2),
 ('301', 3),
 ('452', 1),
 ('265', 1),
 ('213', 2),
 ('96', 1),
 ('387', 1),
 ('246', 2),
 ('286', 4),
 ('270', 1),
 ('230', 1),
 ('1495', 2),
 ('474', 2),
 ('1044', 2),
 ('994', 1),
 ('419', 2),
 ('846', 3),
 ('603', 2),
 ('216', 1),
 ('949', 2),
 ('222', 4),
 ('1039', 1),
 ('1285', 1),
 ('21', 1),
 ('483', 1),
 ('10', 1),
 ('762', 1),
 ('412', 2),
 ('257', 4),
 ('326', 2),
 ('1109', 1),
 ('258', 2),
 ('699', 1),
 ('12', 2),
 ('243', 2),
 ('898', 1),
 ('637', 1),
 ('237', 3),
 ('1160', 1),
 ('134', 2),
 ('571', 2),
 ('56', 2),
 ('70', 2),
 ('433', 1),
 ('154', 2),
 ('130', 1),
 ('91', 1),
 ('778', 1),
 ('111', 3),
 ('993', 1),
 ('506', 2),
 ('824', 1),
 ('66', 2),
 ('1', 2),
 ('234', 2),
 ('150', 1),
 ('588', 1),
 ('208',

In [120]:
type(my_film)

collections.defaultdict

In [121]:
list_of_rating = ['1','2','3','4','5']

In [122]:
dict_answer = {'hist_film':[],'hist_all':[]}
for i in list_of_rating:
    dict_answer['hist_film'].append(my_film[i])
    dict_answer['hist_all'].append(all_films[i])

In [123]:
dict_answer

{'hist_film': [4, 6, 23, 79, 186],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [ ]:
import json

with open('lab01.json', "w") as file:
    json.dump(dict_answer, file)

In [124]:
spark.stop()